# Delete Walls Example

This notebook demonstrates how to delete walls from Revit models using the Autodesk Platform Services (APS) Design Automation API. 

This example follows the official tutorial: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/

**Note:** The input Revit file and compiled app bundle (ZIP) can be found in `examples\Revit_01_delete_walls\files`

<p align="center">
  <img src="./files/cover.png" alt="Cover Image">
</p>

## Step 1: Setup and Import Libraries

Import all necessary libraries and modules.

In [ ]:
import os
import json
import logging
import uuid
from pathlib import Path
from dotenv import load_dotenv

from aps_automation_sdk.classes import (
    Activity,
    ActivityInputParameter,
    ActivityOutputParameter,
    AppBundle,
    WorkItem
)

from aps_automation_sdk.utils import (
    delete_activity,
    delete_appbundle,
    get_token,
    set_nickname
)

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

## Step 2: Load Environment Variables and Configure Authentication

Load credentials from environment variables and get an authentication token.

**Prerequisites:**
1. Create an APS application to get your credentials. Follow these tutorials:
   - Create App: https://aps.autodesk.com/en/docs/oauth/v2/tutorials/create-app
   - Design Automation Setup: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step2-create-forge-app/

2. Create a `.env` file in the project root with your credentials (see `.env.sample` for reference)

**Note on Nicknames:** The nickname must be unique across all APS apps globally. If you encounter errors, try using alphanumeric unique values. Choose a distinctive nickname to avoid conflicts.

In [ ]:
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID", "")
CLIENT_SECRET = os.getenv("CLIENT_SECRET", "")

# Get authentication token
token = get_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

# Set nickname. If the app already has a nickname, the previous one will be returned  like "vitkortest"
nickname = set_nickname(token, "myUniqueNickNameHere")

print(f"Authentication successful. Nickname: {nickname}")

## Step 3: Define Project Constants

Set up all the constants needed for the IFC export workflow.

In [ ]:
# Define constants
app_bundle_name = "DeleteWallsApp"
activity_name = "DeleteWallsActivity"
alias = "prod"
bucket_key = uuid.uuid4().hex
zip_path = Path.cwd() / "files" / "DeleteWallsApp.zip"

# Create full aliases
appbundle_full_alias = f"{nickname}.{app_bundle_name}+{alias}"
activity_full_alias = f"{nickname}.{activity_name}+{alias}"

print(f"App Bundle: {appbundle_full_alias}")
print(f"Activity: {activity_full_alias}")

## Step 4: Create and Deploy App Bundle

Register and upload the app bundle that contains the wall deletion logic.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step4-publish-appbundle/

In [ ]:
bundle = AppBundle(
    appBundleId=app_bundle_name,
    engine="Autodesk.Revit+2024",
    alias=alias,
    zip_path=str(zip_path),
    description="Delete walls from the model",
)

bundle.deploy(token)
print("App bundle deployed successfully!")

## Step 5: Define Activity Parameters

Create input and output parameters for the activity.

**Reference:** https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step5-publish-activity/#step-1-create-a-new-activity

In [ ]:
# Input: Revit file
input_revit = ActivityInputParameter(
    name="rvtFile",
    localName="input.rvt",
    verb="get",
    description="Input Revit File",
    required=True,
    is_engine_input=True,
    bucketKey=bucket_key,
    objectKey="input.rvt",
)

# Output: Zipped IFC files
output_file = ActivityOutputParameter(
    name="result",
    localName="result.rvt",
    verb="put",
    description="Results",
    zip=False,
    bucketKey=bucket_key,
    objectKey="result.rvt",
    required=True,
)

print("Activity parameters defined successfully!")

## Step 6: Create and Deploy Activity

Create the activity that links the app bundle with the parameters.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step5-publish-activity/

In [ ]:
activity = Activity(
    id=activity_name,
    parameters=[input_revit, output_file],
    engine="Autodesk.Revit+2024",
    appbundle_full_name=appbundle_full_alias,
    description="Delete walls from Revit file Updated.",
    alias=alias,
)

# Set the Revit command
activity.set_revit_command_line()
activity.deploy(token=token)
print("Activity deployed successfully!")

## Step 7: Upload Input Revit File

Upload the Revit file to Object Storage Service (OSS).

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step6-prepare-cloud-storage/

In [ ]:
input_rvt_path = Path.cwd() / "files" / "DeleteWalls.rvt"
input_revit.upload_file_to_oss(file_path=str(input_rvt_path), token=token)
print(f"Input Revit file uploaded: {input_rvt_path.name}")

## Step 8: Create and Execute Work Item

Create a work item with all parameters and execute it.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step7-post-workitem/

In [ ]:
work_item = WorkItem(
    parameters=[input_revit, output_file],
    activity_full_alias=activity_full_alias
)

print("Work item created. Starting execution...")

## Step 10: Poll Work Item Status

Monitor the work item execution until completion (max 10 minutes).

In [ ]:
status_resp = work_item.execute(token=token, max_wait=600, interval=10)
last_status = status_resp.get("status", "")

print(f"Work item completed with status: {last_status}")
print(f"Full response: {json.dumps(status_resp, indent=2)}")

## Step 11: Download Results

If successful, download the results.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step7-post-workitem/

In [ ]:
if last_status == "success":
    out_dir = Path.cwd() / "files" / "output"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_zip = out_dir / "output.rvt"
    output_file.download_to(output_path=str(out_zip), token=token)
    print("✅ Download successful: files/output/output.rvt")
else:
    print(f"❌ Work item failed or did not complete successfully. Status: {last_status}")

## Step 12: Cleanup Resources

Delete the activity and app bundle to clean up resources.

In [ ]:
print("Cleaning up: deleting activity and app bundle...")

try:
    delete_activity(activityId=activity_name, token=token)
    print(f"✅ Activity deleted: {activity_full_alias}")
except Exception as e:
    print(f"⚠️ Failed to delete activity: {e}")

try:
    delete_appbundle(appbundleId=app_bundle_name, token=token)
    print(f"✅ App bundle deleted: {app_bundle_name}")
except Exception as e:
    print(f"⚠️ Failed to delete app bundle: {e}")

print("\n🎉 IFC Export workflow completed!")